# Mass-Spring-Damper System: Postprocessing the DeepONet

In [2]:
### Importing Libraries

import sys
print(sys.version)
import os
import time


### Defining WORKSPACE_PATH

# WORKSPACE_PATH = os.environ['WORKSPACE_PATH']
WORKSPACE_PATH = os.path.join(os.getcwd(), '../../../../../../')
ROMNet_fld     = os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/')

### Importing External Libraries

import numpy                             as np
import pandas                            as pd


### Importing Matplotlib and Its Style

import matplotlib.pyplot                 as plt
import matplotlib.cm                     as cm
import matplotlib.animation              as animation

#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/presentation.mplstyle'))
#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/zoomed.mplstyle'))
plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/paper_1column.mplstyle'))
#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/paper_2columns.mplstyle'))


from scipy.integrate import solve_ivp
import pyDOE
from PCAfold         import PCA          as PCAA


import romnet                            as rmnt

3.9.12 (main, Mar 26 2022, 15:51:13) 
[Clang 12.0.0 (clang-1200.0.32.29)]


## Defining Input Data

In [ ]:
#path_to_run_fld = os.path.join(WORKSPACE_PATH, 'ROMNet/Rect_100Instants_TransRotScale/DeepONet/Run_1/')
path_to_run_fld = os.path.join(WORKSPACE_PATH, 'ROMNet/Rect_100Instants_TransRotScale/DeepONet/TestCase1/')
RotFlg      = True
TransFlg    = True
ScaleFlg    = True

FigDir          = os.path.join(WORKSPACE_PATH, '../Desktop/Paper_Figures_DeepONet_TEMP/')


TrainingCases   = [0]#[0,2,4,6,8]
TestCases       = [0,2,4]#[0,2]

NSamples        = 1

LineVec         = ['-',':','--','.-']*10
ColorVec        = ['#190707', '#dd3232', '#0065a9', '#348a00','#985396','#f68b69']

In [ ]:
Theta0  = 0./180*np.pi
w_Theta = 36./180*np.pi

w_Psi = 50./180*np.pi
ca    = 0.5
cx0   = 1.
cy0   = -0.5

lx0   = 8.
ly0   = 6.

xMin    = -15.
xMax    = 15.
yMin    = -15.
yMax    = 15.

v_zz  = 0.2
zz0   = 1.

## Loading ROMNet's Input Data File

In [ ]:
print("\n[ROMNet]: Reading Input File from: ", path_to_run_fld)
sys.path.insert(0, path_to_run_fld)

print("\n[ROMNet]: Keep Loading Modules and Functions...")
from ROMNet_Input import inputdata

print("\n[ROMNet]: Initializing Input ...")
InputData               = inputdata(WORKSPACE_PATH)


InputData.InputFilePath = path_to_run_fld+'/ROMNet_Input.py'
InputData.train_int_flg = 0
InputData.path_to_run_fld  = path_to_run_fld

## Loading Tensorflow's Model for the Network 

In [ ]:
# import tensorflow as tf
# tf.compat.v1.disable_eager_execution()

surrogate_type = InputData.surrogate_type
if (surrogate_type == 'FNN-SourceTerms'):
    surrogate_type = 'FNN'

Net   = getattr(rmnt.architecture, surrogate_type)

model = rmnt.model.Model_TF(InputData)

if (InputData.phys_system is not None):
    System = getattr(rmnt.pinn.system, InputData.phys_system)
    system = System(InputData)
    
model.build(InputData, None, Net, system)#, loadfile_no='000027')

NN               = model.net

trainable_params = np.sum([np.prod(v.get_shape()) for v in NN.trainable_weights])
print('No of trainable_params = ', trainable_params)

## Evaluating DeepONet on Test Data

In [ ]:
input_vars  = model.net.input_vars
trunk_vars  = InputData.input_vars['DeepONet']['Trunk']
branch_vars = InputData.input_vars['DeepONet']['Branch']

In [ ]:
plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/heavy.mplstyle'))

Nx      = 200
Ny      = 200

t0      = 0.
tEnd    = 10.
Nt      = 4

xMin_   = -15.
xMax_   = 15.
yMin_   = -15.
yMax_   = 15.

# = 10.
axx = 10.
bxx = 10.
ayy = 10.
byy = 10.
    
jt      = 0
tVec    = np.linspace(t0,tEnd,Nt)
for it, t in enumerate(tVec):
    print('t=',t)
 
    if (RotFlg == True):
        #Theta = Theta0 + w_Theta*t
        Theta = Theta0 + np.cos(t/10.*(4*np.pi)+2.)*np.pi
    else:
        Theta = Theta0
        
    if (TransFlg == True):
        Psi   = t  * w_Psi
        cr    = ca * Psi
        cx    = cx0 + cr * np.cos(Psi)
        cy    = cy0 + cr * np.sin(Psi)  
    else:
        cx    = 0.
        cy    = 0.
        
    if (ScaleFlg == True):
        s     = (np.sin(t/tEnd*360. / 180.*np.pi)+2)
    else:
        s     = 1.
    
    zz    = zz0 + t*v_zz

    
    Mat       = np.zeros((Nx,Ny))
    InputPred = np.zeros((Nx*Ny,3))
    
    x    = np.linspace(xMin_,xMax_,Nx)
    y    = np.linspace(yMin_,yMax_,Ny)

    i    = 0
    for ix, x_ in enumerate(x):
        for iy, y_ in enumerate(y):
            xrot_      = x_*np.cos(Theta) - y_*np.sin(Theta)
            yrot_      = x_*np.sin(Theta) + y_*np.cos(Theta)
            xrot_      = xrot_-cx
            yrot_      = yrot_-cy
            xrot_      = xrot_*s
            yrot_      = yrot_*s

            zx_1 = (np.tanh((xrot_+lx0)*axx) + np.tanh((lx0-xrot_)*bxx))/2
            zy_1 = (np.tanh((yrot_+ly0)*ayy) + np.tanh((ly0-yrot_)*byy))/2
            
            Mat[ix,iy] = np.exp(zx_1+zy_1)*zz
            
            InputPred[i,0] = t
            InputPred[i,1] = x_
            InputPred[i,2] = y_

            i+=1
            
    InputPred = pd.DataFrame(InputPred, columns=['t','x','y'])
    yMat      = model.predict(InputPred)

    
#     fig, (ax1, ax2) = plt.subplots(1,2,figsize=(12,12))
    
#     ax1.imshow((Mat).reshape(Nx,Ny), animated=True, origin='lower', cmap=cm.turbo, extent=([xMin, xMax, yMin, yMax]))
#     ax1.set_xlabel('x')
#     ax1.set_ylabel('y')
    
#     ax2.imshow((yMat).reshape(Nx,Ny), animated=True, origin='lower', cmap=cm.turbo, extent=([xMin, xMax, yMin, yMax]))
#     ax2.set_xlabel('x')
#     ax2.set_yticks([])

#     fig = plt.figure()
#     plt.imshow((Mat).reshape(Nx,Ny), origin='lower', cmap=cm.turbo, extent=([xMin_, xMax_, yMin_, yMax_]))
#     plt.xlabel('x')
#     plt.ylabel('y')
#     plt.savefig(path_to_run_fld+'/Figures/Rect_Orig_'+str(jt+1)+'.pdf', format='pdf', bbox_inches='tight')
#     #plt.savefig(path_to_run_fld+'/Figures/Rect_Orig_Supp_'+str(jt+1)+'.pdf', format='pdf', bbox_inches='tight')
#     #plt.savefig(path_to_run_fld+'/Figures/XRec_t'+str(t)+'_Orig.pdf', format='pdf', bbox_inches='tight')
    
    fig, ax = plt.subplots()
    ax.imshow((yMat).reshape(Nx,Ny), origin='lower', cmap=cm.turbo, extent=([xMin_, xMax_, yMin_, yMax_]))
    rect = plt.Rectangle((xMin,yMin), xMax-xMin, yMax-yMin, linewidth=2, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    plt.savefig(path_to_run_fld+'/Figures/Rect_flexDeepONet_Supp_'+str(jt+1)+'.pdf', format='pdf', bbox_inches='tight')
    #plt.savefig(path_to_run_fld+'/Figures/Rect_DeepONet_Supp_'+str(jt+1)+'.pdf', format='pdf', bbox_inches='tight')
    #plt.savefig(path_to_run_fld+'/Figures/XRec_t'+str(t)+'_DeepOnet.pdf', format='pdf', bbox_inches='tight')
    
    jt+=1

In [ ]:
plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/paper_1column.mplstyle'))

Nx      = 200
Ny      = 200

t0      = 0.
tEnd    = 10.
Nt      = 100

xMin_   = -15.
xMax_   = 15.
yMin_   = -15.
yMax_   = 15.

# = 10.
axx = 10.
bxx = 10.
ayy = 10.
byy = 10.


tVec = np.linspace(0,10,4)
for it, t in enumerate(tVec):
    print('t=', t)
    

    if (RotFlg == True):
        #Theta = Theta0 + w_Theta*t
        Theta = Theta0 + np.cos(t*4./10.*(np.pi)+2.)*np.pi
    else:
        Theta = Theta0

    if (TransFlg == True):
        Psi   = t*w_Psi
        cr    = ca * Psi
        cx    = cx0 + cr * np.cos(Psi)
        cy    = cy0 + cr * np.sin(Psi)  
    else:
        cx    = 0.
        cy    = 0.

    if (ScaleFlg == True):
        s    = np.sin(t/tEnd*360. / 180.*np.pi)+2.
    else:
        s     = 1

    
    zz    = zz0 + t*v_zz

    
    InputPred = np.zeros((Nx,3))
    yData     = np.zeros(Nx)
    x  = np.linspace(-10,10,Nx)
    y_ = 0.
    i  = 0
    for ix, x_ in enumerate(x):
        InputPred[i,0] = t
        InputPred[i,1] = x_
        InputPred[i,2] = y_

        
        xrot_          = x_*np.cos(Theta) - y_*np.sin(Theta)
        yrot_          = x_*np.sin(Theta) + y_*np.cos(Theta)
        xrot_          = xrot_-cx
        yrot_          = yrot_-cy
        xrot_          = xrot_*s
        yrot_          = yrot_*s
        zx_1           = (np.tanh((xrot_+lx0)*axx) + np.tanh((lx0-xrot_)*bxx))/2
        zy_1           = (np.tanh((yrot_+ly0)*ayy) + np.tanh((ly0-yrot_)*byy))/2
        
        
#         xrot_          = x_*np.cos(Theta) + y_*np.sin(Theta)
#         yrot_          = x_*np.sin(Theta) - y_*np.cos(Theta)
        
#         zx             = np.tanh((xrot_-cx+lx/2)*10.) - np.tanh((xrot_-cx-lx/2)*10.)
#         zy             = np.tanh((yrot_-cy+ly/2)*10.) - np.tanh((yrot_-cy-ly/2)*10.) 

        yData[i]       = np.exp(zx_1+zy_1)*zz
        i+=1
    
            
    InputPred = pd.DataFrame(InputPred, columns=['t','x','y'])
    
    yMat = model.predict(InputPred)
    
    plt.figure()
    plt.plot(x, yData, color=ColorVec[it])
    plt.plot(x, yMat, ':', color=ColorVec[it])
    plt.xlabel('x')
    plt.ylabel('z')
    plt.savefig(path_to_run_fld+'/Figures/XSlice_t'+str(t)+'_DeepOnet.pdf', format='pdf', bbox_inches='tight')

In [ ]:
Nx      = 200
Ny      = 200

t0      = 0.
tEnd    = 10.
Nt      = 100

xMin_   = -15.
xMax_   = 15.
yMin_   = -15.
yMax_   = 15.

# = 10.
axx = 10.
bxx = 10.
ayy = 10.
byy = 10.


tVec = np.linspace(0,10,100)
for it, t in enumerate(tVec):
    #print('t=', t)
    

    if (RotFlg == True):
        #Theta = Theta0 + w_Theta*t
        Theta = Theta0 + np.cos(t*4./10.*(np.pi)+2.)*np.pi
    else:
        Theta = Theta0

    if (TransFlg == True):
        Psi   = t*w_Psi
        cr    = ca * Psi
        cx    = cx0 + cr * np.cos(Psi)
        cy    = cy0 + cr * np.sin(Psi)  
    else:
        cx    = 0.
        cy    = 0.

    if (ScaleFlg == True):
        s    = np.sin(t/tEnd*360. / 180.*np.pi)+2.
    else:
        s     = 1

    
    zz    = zz0 + t*v_zz

    
    InputPred = np.zeros((Nx,3))
    yData     = np.zeros(Nx)
    x  = np.linspace(-10,10,Nx)
    y_ = 0.
    i  = 0
    for ix, x_ in enumerate(x):
        InputPred[i,0] = t
        InputPred[i,1] = x_
        InputPred[i,2] = y_

        
        xrot_          = x_*np.cos(Theta) - y_*np.sin(Theta)
        yrot_          = x_*np.sin(Theta) + y_*np.cos(Theta)
        xrot_          = xrot_-cx
        yrot_          = yrot_-cy
        xrot_          = xrot_*s
        yrot_          = yrot_*s
        zx_1           = (np.tanh((xrot_+lx0)*axx) + np.tanh((lx0-xrot_)*bxx))/2
        zy_1           = (np.tanh((yrot_+ly0)*ayy) + np.tanh((ly0-yrot_)*byy))/2
    

        yData[i]       = np.exp(zx_1+zy_1)*zz
        i+=1
            
    InputPred = pd.DataFrame(InputPred, columns=['t','x','y'])
    
    yMat      = model.predict(InputPred)
    
    
    
    if (it == 0):   
        MSE  = (yMat[:,0] - yData)**2#/y_Data[1::]**2
    else:
        MSE += (yMat[:,0] - yData)**2#/y_Data[1::]**2
        
        
    
MSE_ = MSE.sum(axis=0) / (it) / (MSE.shape[0])
print('Mean Squared Error = ', MSE_)

RMSE = np.sqrt( MSE.sum(axis=0) / ((it) * MSE.shape[0]) )
print('Root Mean Squared Error = ', RMSE)

In [ ]:
# plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/video.mplstyle'))

# img    = [] # some array of images
# frames = [] # for storing the generated images
# fig    = plt.figure(figsize=(10,8))

# Nx      = 200
# Ny      = 200

# t0      = 0.
# tEnd    = 10.
# Nt      = 100
    
# tVec    = np.linspace(t0,tEnd,Nt)
# for it, t in enumerate(tVec):
#     print('t=',t)

#     InputPred = np.zeros((Nx*Ny,3))
    
#     x    = np.linspace(xMin,xMax,Nx)
#     y    = np.linspace(yMin,yMax,Ny)

#     i    = 0
#     for ix, x_ in enumerate(x):
#         for iy, y_ in enumerate(y):                        
#             InputPred[i,0] = t
#             InputPred[i,1] = x_
#             InputPred[i,2] = y_

#             i+=1
            
#     InputPred = pd.DataFrame(InputPred, columns=['t','x','y'])
#     yMat      = model.predict(InputPred)

#     im     = plt.imshow((yMat).reshape(Nx,Ny), animated=True, origin='lower', cmap=cm.turbo, extent=([xMin, xMax, yMin, yMax]))
#     plt.xlabel('x')
#     plt.ylabel('y')
#     frames.append([im])
    
# ani = animation.ArtistAnimation(fig, frames, interval=5000, blit=True, repeat_delay=50000)
# writergif = animation.PillowWriter(fps=30)
# ani.save(path_to_run_fld+'/Figures/Video.gif',writer=writergif)

In [ ]:
# plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/video.mplstyle'))

# img     = [] # some array of images
# frames  = [] # for storing the generated images
# fig     = plt.figure(figsize=(20,8))
# axs     = []
# axs.append( fig.add_subplot(1,2,1) )
# axs.append( fig.add_subplot(1,2,2) )

# #axs     = plt.figure(constrained_layout=True, figsize=(10,8)).subplots(1, 2, sharey=True)

# Nx      = 200
# Ny      = 200

# t0      = 0.
# tEnd    = 10.
# Nt      = 100

# xMin_   = -15.
# xMax_   = 15.
# yMin_   = -15.
# yMax_   = 15.

# # = 10.
# axx = 10.
# bxx = 10.
# ayy = 10.
# byy = 10.
    
    
# tVec    = np.linspace(t0,tEnd,Nt)
# for it, t in enumerate(tVec):
#     print('t=',t)
    
    
#     if (RotFlg == True):
#         #Theta = Theta0 + w_Theta*t
#         Theta = Theta0 + np.cos(t/10.*(4*np.pi)+2.)*np.pi
#     else:
#         Theta = Theta0

#     if (TransFlg == True):
#         Psi   = t  * w_Psi
#         cr    = ca * Psi
#         cx    = cx0 + cr * np.cos(Psi)
#         cy    = cy0 + cr * np.sin(Psi)  
#     else:
#         cx    = 0.
#         cy    = 0.

#     if (ScaleFlg == True):
#         s     = (np.sin(t/tEnd*360. / 180.*np.pi)+2)
#     else:
#         s     = 1.

#     zz    = zz0 + t*v_zz


#     Mat       = np.zeros((Nx,Ny))
#     InputPred = np.zeros((Nx*Ny,3))
    
#     x    = np.linspace(xMin,xMax,Nx)
#     y    = np.linspace(yMin,yMax,Ny)

#     i    = 0
#     for ix, x_ in enumerate(x):
#         for iy, y_ in enumerate(y):
#             InputPred[i,0] = t
#             InputPred[i,1] = x_
#             InputPred[i,2] = y_
            
#             xrot_          = x_*np.cos(Theta) - y_*np.sin(Theta)
#             yrot_          = x_*np.sin(Theta) + y_*np.cos(Theta)
#             xrot_          = xrot_-cx
#             yrot_          = yrot_-cy
#             xrot_          = xrot_*s
#             yrot_          = yrot_*s
#             zx_1           = (np.tanh((xrot_+lx0)*axx) + np.tanh((lx0-xrot_)*bxx))/2
#             zy_1           = (np.tanh((yrot_+ly0)*ayy) + np.tanh((ly0-yrot_)*byy))/2

#             Mat[ix,iy]     = np.exp(zx_1+zy_1)*zz
#             i+=1
            
#     InputPred = pd.DataFrame(InputPred, columns=['t','x','y'])
#     yMat      = model.predict(InputPred)
    

#     im1       = axs[0].imshow((Mat).reshape(Nx,Ny), animated=True, origin='lower', cmap=cm.turbo, extent=([xMin, xMax, yMin, yMax]))
    
#     im2       = axs[1].imshow((yMat).reshape(Nx,Ny), animated=True, origin='lower', cmap=cm.turbo, extent=([xMin, xMax, yMin, yMax]))
#     rect      = plt.Rectangle((-10.,-10.), 20., 20., linewidth=2, edgecolor=ColorVec[1], facecolor='none')
#     axs[1].add_patch(rect)
#     axs[1].text( -9.8,  8.5, r'Test Predictions', color=ColorVec[1], fontsize=26, weight='bold')
#     axs[1].text(-14.8, 13.5, r'Extrapolations', color=ColorVec[1], fontsize=26, weight='bold')

#     axs[0].set_title("Rigid Body Dynamics", pad=10)
#     axs[0].set_xlabel('x')
#     axs[0].set_ylabel('y')
    
#     axs[1].set_title("flexDeepONet Architecture,\n 1,910 Trainable Parameters", pad=10)
#     axs[1].set_xlabel('x')
#     axs[1].set_yticklabels([])
    
#     frames.append([im1,im2])
    
# ani = animation.ArtistAnimation(fig, frames, interval=5000, blit=True, repeat_delay=50000)
# writergif = animation.PillowWriter(fps=30)
# ani.save(path_to_run_fld+'/Figures/Video.gif',writer=writergif)

## Plotting Loss Histories

In [ ]:
plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/paper_1column.mplstyle'))

Data = pd.read_csv(path_to_run_fld+'/Training/History.csv')

fig  = plt.figure(figsize=(12,8))
plt.plot(Data['tot_loss'],     label='Training')
plt.plot(Data['val_tot_loss'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Tot. Loss [MSE]')
plt.legend()
plt.yscale('log')


fig  = plt.figure(figsize=(12,8))
plt.plot(Data['pts_loss'],     label='Training')
plt.plot(Data['val_pts_loss'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Data Loss [MSE]')
plt.legend()
plt.yscale('log')
#plt.ylim([1.e-2, 1.e0])

# fig  = plt.figure(figsize=(12,8))
# plt.plot(Data['res_loss'],     label='Training')
# plt.plot(Data['val_res_loss'], label='Validation')
# plt.xlabel('Epoch')|
# plt.ylabel('Residual Loss [MSE]')
# plt.legend()
# plt.yscale('log')

# fig  = plt.figure(figsize=(12,8))
# plt.plot(Data['ics_loss'],     label='Training')
# plt.plot(Data['val_ics_loss'], label='Validation')
# plt.xlabel('Epoch')
# plt.ylabel('I.C.s Loss [MSE]')
# plt.legend()
# plt.yscale('log')

## Evaluating Trunk

In [ ]:
# plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/heavy.mplstyle'))

# Nx   = 200
# Ny   = 200

# x    = np.linspace(xMin*2,xMax*2,Nx)
# y    = np.linspace(yMin*2,yMax*2,Ny)

# InputData = np.zeros((Nx*Ny,2))
# i         = 0
# for ix in range(Nx):
#     x_ = x[ix]
#     for iy in range(Ny):
#         InputData[i,0] = x_
#         InputData[i,1] = y[iy]
#         i+=1

# output_trunk = model.net.system_of_components['DeepONet'].components['Trunk'].call(InputData, model.net.layers_dict, None, training=False)

# Mode = np.zeros((Nx,Ny))
# i    = 0
# for ix in range(Nx):
#     for iy in range(Ny):
#         Mode[ix,iy] = output_trunk[i,0]
#         i+=1
        
# plt.imshow(Mode, animated=True, origin='lower', cmap=cm.turbo, extent=([xMin*2, xMax*2, yMin*2, yMax*2]))
# plt.xlabel('x')
# plt.ylabel('y')
# plt.savefig(path_to_run_fld+'/Figures/Mode1_DeepOnet.pdf', format='pdf', bbox_inches='tight')

## Evaluating Rigid Block

In [ ]:
t0      = 0.
tEnd    = 10.

In [ ]:
plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/paper_1column.mplstyle'))
plt.figure()

tVec = np.linspace(0,10,100)[...,np.newaxis]

output_shift    = model.net.system_of_components['DeepONet'].components['Shift'].call(tVec, model.net.layers_dict, None, training=False)
output_stretch  = model.net.system_of_components['DeepONet'].components['Stretch'].call(tVec, model.net.layers_dict, None, training=False)
output_rotation = model.net.system_of_components['DeepONet'].components['Rotation'].call(tVec, model.net.layers_dict, None, training=False)

In [ ]:
tEnd  = 10.
s    = (np.sin(tVec/tEnd*360. / 180.*np.pi)+2)
    
plt.plot(tVec,s)

plt.plot(tVec, output_stretch)

plt.xlabel('Time [s]')
plt.ylabel(r'Scaling')

In [ ]:
Theta   = Theta0 + np.cos(tVec/10.*(4*np.pi)+2.)*np.pi

plt.plot(tVec,Theta/np.pi*180.)
plt.plot(tVec, output_rotation/np.pi*180.+(Theta[-1]-output_rotation[-1])/np.pi*180.)
plt.xlabel('Time [s]')
plt.ylabel(r'$\theta$ [deg]')

In [ ]:
Theta   = Theta0 + np.cos(tVec/10.*(4*np.pi)+2.)*np.pi

plt.plot(tVec, np.cos(Theta))
plt.plot(tVec, np.cos(output_rotation))#+(Theta[-1]-output_rotation[-1])))
plt.xlabel('Time [s]')
plt.ylabel(r'$cos(\theta)$')

In [ ]:
Psi   = tVec*w_Psi
cr    = ca * Psi
cx    = cx0 + cr * np.cos(Psi)
cy    = cy0 + cr * np.sin(Psi)  
plt.plot(tVec, cx, '-', color=ColorVec[0], label='$x_c$')
plt.plot(tVec, cy, '-', color=ColorVec[1], label='$y_c$')

sca   = 1#-output_1[:,3]
plt.plot(tVec, output_shift[:,0]*sca, ':', color=ColorVec[0], label=None)
plt.plot(tVec, output_shift[:,1]*sca, ':', color=ColorVec[1], label=None)

plt.xlabel('Time [s]')
plt.ylabel(r'Center Coordinates')
plt.legend()

In [ ]:
Psi   = tVec*w_Psi
cr    = ca * Psi
cx    = cx0 + cr * np.cos(Psi)
cy    = cy0 + cr * np.sin(Psi)  

sca   = 1#-1/output_1[:,3]
plt.plot(cx, cy, '-', color=ColorVec[0], label=None)
plt.plot(output_shift[:,0]*sca, output_shift[:,1]*sca, ':', color=ColorVec[0], label=None)


plt.xlabel('Time [s]')
plt.ylabel(r'Center Coordinates')
plt.legend()